In [6]:
import tensorflow as tf
import argparse
import logging
import sys

import pyphen
import nltk


logger = logging.getLogger()
logger.setLevel(logging.INFO)
console_out = logging.StreamHandler(sys.stdout)
console_out.setLevel(logging.DEBUG)
logger.addHandler(console_out)

In [27]:
def parse_arguments():
    """
    commandline arguments parser
    return text to be cleaned
    """
    parser = argparse.ArgumentParser(
        description="수정할 텍스트를 입력합니다"
    )
    
    parser.add_argument('text', metavar='input text', type=str)
    args = parser.parse_args()
    return args.text

In [5]:
def clean_input(text):
    """
    text cleaning function
    parameter text: user input text
    delete characters not ascii, and return text
    """
    return str(text.encode().decode('ascii', errors='ignore'))

In [7]:
def preprocess_input(text):
    """
    tokenize cleaned text
    parameter text: cleaned text
    return tokenized text
    """
    sentences = nltk.sent_tokenize(text)
    tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    return tokens

In [8]:
def compute_average_word_length(tokens):
    """
    한 문장에 있는 단어의 길이를 계산
    parameter tokens: 단어 리스트
    return: tokens에 있는 단어의 평균 길이
    """
    word_lengths = [len(word) for word in tokens]
    return sum(word_lengths)/len(word_lengths)


def compute_total_average_word_length(sentence_list):
    """
    여러 문장에 대한 단어의 평균 길이 계산
    param sentence_list: 단어의 리스트로 구성된 문장 리스트
    return: sentence_list에 있는 문장의 단어의 평균 길이
    """
    lengths = [compute_average_word_length(tokens) for tokens in sentence_list]
    return sum(lengths)/len(lengths)

In [9]:
def compute_total_unique_words_fraction(sentence_list):
    """
    고유한 단어의 비율을 계산
    param sentence_list: 단어의 리스트로 구성된 문장 리스트
    return: 고유한 단어의 비율
    """
    all_words = [word for word_list in sentence_list for word in word_list]
    unique_words = set(all_words)
    return len(unique_words)/len(all_words)

In [10]:
def count_word_usage(tokens, word_list):
    """
    주어진 단어 리스트의 등장 횟수
    param tokens: 문장의 단어 리스트
    param word_list: 탐색하려는 단어의 리스트
    return: 주어진 단어 리스트의 등장 횟수
    """
    return len([word for word in tokens if word.lower() in word_list])

In [12]:
def count_word_syllabels(word):
    """
    단어에 있는 음절의 개수를 셈
    param word: 단어
    return 단어에 있는 음절의 수
    """
    dic = pyphen.Pyphen(lang="en_US")
    hyphenated = dic.inserted(word)
    return len(hyphenated.split("-"))


def count_sentene_syllabels(tokens):
    """
    문장에 있는 음절의 개수를 셈
    param tokens: 단어와 구둣점의 리스트
    return: 문장에 있는 음절의 수
    """
    punctuation = ".,!?/"
    return sum(
        [
            count_word_syllabels(word)
            for word in tokens
            if word not in punctation
        ]
    )


def count_total_syllabels(sentence_list):
    """
    문장 리스트에 있는 음절의 개수를 셈
    param sentence_list: 단어의 리스트로 구성된 문장 리스트
    return: 문장 리스트에 있는 음절의 개수
    """
    return sum(
        [
            count_sentecne_syllabels(tokens)
            for tokens in sentence_list
        ]
    )


def count_words_per_sentence(sentence_tokens):
    """
    문장에 있는 단어의 개수를 셈
    param sentence_tokens: 단어와 구둣점의 리스트
    return: 문장에 있는 단어의 수
    """
    punctuation = ".,!?/"
    return len([word for word in sentence_tokens if word not in punctuation])


def count_total_words(sentence_list):
    """
    총 단어의 개수를 셈
    param sentence_list: 단어의 리스트로 구성된 문장 리스트
    return:총 단어의 개수
    """
    return sum([count_words_per_sentence(tokens) for tokens in sentence_list])

In [13]:
def compute_flesch_reading_ease(total_syllabels, total_words, total_sentences):
    """
    요약 통계로부터 가독성 점수를 계산
    return a readibility score: 점수가 낮을수록 더 복잡한 텍스트임
    """
    return (206.85 - 1.015*(total_words/total_sentences) - 84.6*(total_syllabels/total_words))


def get_reading_level_from_flesch(flesch_score):
    """
    https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests 에서 가져온 임곗값
    :param flesch_score:
    :return: 플레시 점수에 대한 가독성 수준
    """
    if flesch_score < 30:
        return "매우 읽기 어려움"
    elif flesch_score < 50:
        return "읽기 어려움"
    elif flesch_score < 60:
        return "약간 읽기 어려움"
    elif flesch_score < 70:
        return "보통"
    elif flesch_score < 80:
        return "약간 읽기 쉬움"
    elif flesch_score < 90:
        return "읽기 쉬움"
    else:
        return "매우 읽기 쉬움"

In [14]:
def get_suggestions(sentence_list):
    told_said_usage = sum(
        (count_word_usage(tokens, ["told", "said"]) for tokens in sentence_list)
    )
    but_and_usage = sum(
        (count_word_usage(tokens, ["but", "and"]) for tokens in sentence_list)
    )
    wh_adverbs_usage = sum(
        (
            count_word_usage(
                tokens,
                [
                    "when",
                    "where",
                    "why",
                    "whence",
                    "whereby",
                    "wherein",
                    "whereupon",
                ],
            )
            for tokens in sentence_list
        )
    )
    result_str = ""
    adverb_usage = "단어 사용량: %s told/said, %s but/and, %s wh-접속사" % (told_said_usage, but_and_usage, wh_adverbs_usage,)
    result_str += adverb_usage
    
    average_word_length = compute_total_average_word_length(sentence_list)
    unique_words_fraction = compute_total_unique_words_fraction(sentence_list)
    
    word_stats = "평균 단어 길이 %.2f, 고유한 단어의 비율 %.2f" % (average_word_length, unique_words_fraction,)
    
    result_str += "<br/>"
    result_str += word_stats
    
    number_of_syllabels = count_total_syllabels(sentence_list)
    number_of_words = count_total_words(sentence_list)
    number_of_sentences = len(sentence_list)
    
    syllabels_counts = "%d개 음절, %d개 단어, %d개 문장" % (number_of_syllabels, number_of_words, number_of_sentences,)
    
    result_str += "<br/>"
    result_str += syllabels_counts
    
    flesch_score = compute_flesch_reading_ease(number_of_syllabels, number_of_words, number_of_sentences)
    flesch = "플레시 점수 %.2f: %s" % (flesch_score, get_reading_level_from_flesch(flesch_score),)
    
    result_str += "<br/>"
    result_str += flesch
    
    return result_str

In [16]:
def get_recommendations_from_input(txt):
    """
    입력 문자열에 대한 정제, 전처리하고 경험 규칙 기반의 추천을 생성
    return: 텍스트 입력에 대한 추천
    """
    processed = clean_input(txt)
    tokenized_sentences = preprocess_input(processed)
    suggestions = get_suggestions(tokenized_sentences)
    return suggestions

In [24]:
if __name__=="__main__":
    input_text = parse_arguments()
    print(get_recommendations_from_input(input_text))

usage: ipykernel_launcher.py [-h] input text
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2